### Imports

In [1]:
import numpy as np
import scipy as sp
from itertools import combinations
import matplotlib.pyplot as plt

### Functions

In [2]:
def basis_set_generator(tot_sites, N_e_up, N_e_down):
    spin_up_basis = []
    spin_down_basis = []

    for comb_up in combinations(range(tot_sites), N_e_up):
        state = [0] * tot_sites
        for idx in comb_up:
            state[idx] = 1
        spin_up_basis.append(state)

    for comb_down in combinations(range(tot_sites), N_e_down):
        state = [0] * tot_sites
        for idx in comb_down:
            state[idx] = 1
        spin_down_basis.append(state)

    basis = []

    for up_state in spin_up_basis:
        for down_state in spin_down_basis:
            basis.append(up_state + down_state)
    
    return np.array(basis)



# @jit(nopython=True, parallel=True)
# @njit
def creation_operator(state, site):
    
    res_state = state.copy()
    
    if state[site] == 1:
        return 0, None
    
    sign = 1
    for i in range(site):
        if state[i] == 1:
            sign = sign * -1
    
    res_state[site] = 1
    return sign, res_state



# @jit(nopython=True, parallel=True)
# @njit
def annihilation_operator(state, site):
    
    res_state = state.copy()
    
    if state[site] == 0:
        return 0, None
    
    sign = 1
    for i in range(site):
        if state[i] == 1:
            sign = sign * -1
    
    res_state[site] = 0
    return sign, res_state



# @jit(nopython=True, parallel=True)
# @njit
def hopping_operator(state, create_site, destroy_site):

    if (state[create_site] == 1) and (state[destroy_site] == 0):
        return 0, None

    sign_destroy, res_state = annihilation_operator(state, destroy_site)
    
    sign_create, res_state = creation_operator(res_state, create_site)

    return (sign_create * sign_destroy), res_state



# @jit(nopython=True, parallel=True)
# @njit
def state_idx_mapping(basis_set):
    return {tuple(state) : i for i, state in enumerate(basis_set)}



# @jit(nopython=True, parallel=True)
# @njit
def hamiltonian_matrix_generator(basis_set, tot_sites, J_11, J_1, J_33, J_3, U):
    dim = len(basis_set)
    hamiltonian = np.zeros((dim, dim), dtype=np.float64)
    state_idx_dict = state_idx_mapping(basis_set)

    for i, state in enumerate(basis_set):
        for j in range(tot_sites):
            for k in [0, tot_sites]:
                l = j + k
                # print(l)

                if state[l] == 0:
                    continue

                if (l + 1) < (tot_sites + k): # J_11 & J_1 terms
                    if state[l + 1] == 0:
                        destroy_site = l
                        create_site = l + 1
                        # print(initial_position, target_position)
                        sign, res_state = hopping_operator(state, create_site, destroy_site)
                        # print(sign, res_state)

                        # print(f"{state} ---- {destroy_site} ---- {create_site} ---- {sign} ---- {res_state}")

                        if sign == 0:
                            continue

                        res_state_idx = state_idx_dict[tuple(res_state)]
                        # print(res_state_idx)

                        if l % 2 == 0:
                            hamiltonian[res_state_idx, i] += J_11 * sign
                        else:
                            hamiltonian[res_state_idx, i] += J_1 * sign
                
                if l - 1 >= k: # Hermitian conjugate of J_11 & J_1 terms
                    if state[l - 1] == 0:
                        destroy_site = l
                        create_site = l - 1
                        # print(initial_position, target_position)
                        sign, res_state = hopping_operator(state, create_site, destroy_site)
                        # print(sign, res_state)

                        # print(f"{state} ---- {destroy_site} ---- {create_site} ---- {sign} ---- {res_state}")

                        if sign == 0:
                            continue

                        res_state_idx = state_idx_dict[tuple(res_state)]
                        # print(res_state_idx)

                        if l % 2 == 0:
                            hamiltonian[res_state_idx, i] += J_1 * sign
                        else:
                            hamiltonian[res_state_idx, i] += J_11 * sign
                
                if (l + 3) < (tot_sites + k): # J_33 & J_3 terms
                    if state[l + 3] == 0:
                        destroy_site = l
                        create_site = l + 3
                        # print(initial_position, target_position)
                        sign, res_state = hopping_operator(state, create_site, destroy_site)
                        # print(sign, res_state)

                        # print(f"{state} ---- {destroy_site} ---- {create_site} ---- {sign} ---- {res_state}")

                        if sign == 0:
                            continue

                        res_state_idx = state_idx_dict[tuple(res_state)]
                        # print(res_state_idx)

                        if l % 2 == 0:
                            hamiltonian[res_state_idx, i] += J_33 * sign
                        else:
                            hamiltonian[res_state_idx, i] += J_3 * sign
                
                if (l - 3) >= k: # Hermitian conjugate of J_33 & J_3 terms
                    if state[l - 3] == 0:
                        destroy_site = l
                        create_site = l - 3
                        # print(initial_position, target_position)
                        sign, res_state = hopping_operator(state, create_site, destroy_site)
                        # print(sign, res_state)

                        # print(f"{state} ---- {destroy_site} ---- {create_site} ---- {sign} ---- {res_state}")

                        if sign == 0:
                            continue

                        res_state_idx = state_idx_dict[tuple(res_state)]
                        # print(res_state_idx)

                        if l % 2 == 0:
                            hamiltonian[res_state_idx, i] += J_3 * sign
                        else:
                            hamiltonian[res_state_idx, i] += J_33 * sign
    
    for i, state in enumerate(basis_set):
        for j in range(tot_sites):
            if state[j] == state[j + tot_sites] and state[j] == 1:
                hamiltonian[i, i] += U
    
    return hamiltonian



# @njit
def normalize(state):
    return state / (np.linalg.norm(state))



def state_to_position_space(state, basis_set, tot_sites, dim):
    pos_sp_state = np.zeros((tot_sites))

    for i in range(dim):
        pos_sp_state += basis_set[i][0 : tot_sites] * state[i]
        pos_sp_state += basis_set[i][tot_sites : 2 * tot_sites] * state[i]
    
    return normalize(pos_sp_state)

### Simulation

#### System Defination

In [3]:
tot_sites = 8
N_e_up = 4
N_e_down = N_e_up
J_11 = 0.1
J_1 = 0.3
J_33 = 0.1
J_3 = 1
U = 10

#### Basis and Hamiltonian Creation

In [4]:
basis_set = basis_set_generator(tot_sites, N_e_up, N_e_down)

hamiltonian = hamiltonian_matrix_generator(basis_set, tot_sites, J_11, J_1, J_33, J_3, U)

#### Diagonalization

In [5]:
e_val_arr, e_vec_arr = np.linalg.eigh(hamiltonian)
e_vec_arr_transpose = e_vec_arr.transpose()

### Archives & Tests

In [11]:
E = np.linalg.inv(e_vec_arr) @ hamiltonian @ e_vec_arr
dim = len(basis_set)

for i in range(dim):
    print(np.round(e_val_arr[i], decimals=5), end='|')
print()
for i in range(dim):
    print(np.round(E[i, i], decimals=5), end='|')

for i in range(dim):
    if np.round(e_val_arr[i], decimals=5) != np.round(E[i, i], decimals=5):
        print(False)
        break

-0.80917|-0.80911|-0.80507|-0.80505|-0.80504|-0.80502|-0.44935|-0.44861|-0.44479|-0.44301|-0.44263|-0.44178|-0.44163|-0.44022|-0.43842|-0.43677|-0.42991|-0.42977|-0.42105|-0.41912|-0.41826|-0.41651|-0.41635|-0.41511|-0.41408|-0.4139|-0.41301|-0.41199|-0.41015|-0.40821|-0.40298|-0.40086|-0.40006|-0.39801|-0.0788|-0.07869|-0.07284|-0.07068|-0.06854|-0.06667|-0.05389|-0.05289|-0.05221|-0.05166|-0.05146|-0.05063|-0.05019|-0.04951|-0.04599|-0.04295|-0.04178|-0.03898|-0.03874|-0.03567|-0.03553|-0.03549|-0.03374|-0.03255|-0.03072|-0.03053|-0.02896|-0.0269|-0.02278|-0.01964|-0.01807|-0.01483|-0.00934|-0.00526|-0.00404|-0.0|7.72889|7.72961|7.76102|7.76127|7.76679|7.77033|7.77423|7.7781|7.7807|7.78506|7.7884|7.79087|7.80793|7.80995|7.81099|7.81131|7.81316|7.81408|7.81677|7.81922|7.83716|7.83951|7.84089|7.84248|7.84375|7.84489|7.84627|7.8468|7.84761|7.84914|7.85061|7.85209|7.85376|7.85412|7.85508|7.8551|7.85613|7.85999|7.86077|7.86855|8.27571|8.27627|8.30446|8.30719|8.30939|8.31215|8.36242|8.363|

In [7]:
# print(len(e_val_arr))
# print(e_val_arr)

# count = 0
# state_idx = 0
# for i, e in enumerate(np.abs(e_val_arr)):
#     if e < 10**(-10):
#         # state_idx = i
#         print(i, e)
#         count += 1
# # print(count)

# state_ps = state_to_position_space(e_vec_arr[state_idx], basis_set, tot_sites, len(basis_set))

for i, e in enumerate(e_val_arr):
    print(f'{i}---------- {e}')

0---------- -0.8091716318211778
1---------- -0.8091081578657529
2---------- -0.8050734802069757
3---------- -0.8050471069676504
4---------- -0.8050393522538241
5---------- -0.8050176239404448
6---------- -0.44935041449283125
7---------- -0.4486079695805856
8---------- -0.44478656911663106
9---------- -0.4430084569345568
10---------- -0.442629803332717
11---------- -0.4417787084248454
12---------- -0.44162639137555165
13---------- -0.440223213020696
14---------- -0.4384218457565748
15---------- -0.4367685684683733
16---------- -0.4299114408739135
17---------- -0.4297669887569707
18---------- -0.42105430005294714
19---------- -0.4191223490849821
20---------- -0.41826399136002135
21---------- -0.4165063168756564
22---------- -0.4163475385498372
23---------- -0.4151110354702846
24---------- -0.4140788908898995
25---------- -0.4139028906707185
26---------- -0.4130058272670707
27---------- -0.41198989030244443
28---------- -0.4101510722003476
29---------- -0.40820815752898554
30---------- -0

In [8]:
plt.plot(list(range(tot_sites)), state_ps**2)
plt.show()

NameError: name 'state_ps' is not defined